# Let's build a human-face detector using state-of-the-art YOLOv8

In [ ]:
# Importing necessary libraries
import os
import numpy as np
import pandas as pd
import shutil
import cv2
import random
import matplotlib.pyplot as plt
import copy
import wandb

You will need a unique API key to log in to Weights & Biases. 

1. If you don't have a Weights & Biases account, you can go to https://wandb.ai/site and create a FREE account.
2. Access your API key: https://wandb.ai/authorize.

There are two ways you can login using a Kaggle kernel:

1. Run a cell with `wandb.login()`. It will ask for the API key, which you can copy + paste in.
2. You can also use Kaggle secrets to store your API key and use the code snippet below to login. Check out this [discussion post](https://www.kaggle.com/product-feedback/114053) to learn more about Kaggle secrets. 



In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api")

In [ ]:
wandb.login(key=secret_value_0)

#### Defining some variables which will be needed later

In [ ]:
bs=' ' # blank-space
class_id=0 # id for face
newline='\n' # new line character
extension='.txt' # extension for text file

##### Separating paths for training, validation and test datasets for both images and labels

In [ ]:
# Creating paths for separate images and labels
curr_path=os.getcwd()
imgtrainpath = os.path.join(curr_path,'images','train')
imgvalpath=os.path.join(curr_path,'images','validation')
imgtestpath=os.path.join(curr_path,'images','test')

labeltrainpath=os.path.join(curr_path,'labels','train')
labelvalpath=os.path.join(curr_path,'labels','validation')
labeltestpath=os.path.join(curr_path,'labels','test')

##### Labels path is where all labels will be stored first before dividing them in train, validation and test

In [ ]:
# Defining data path and labels_path
data_path='/kaggle/input/human-faces-object-detection'
labels_path = os.path.join(curr_path, 'face_labels')


In [ ]:
# Creating labels path
os.makedirs(labels_path)

# Checking Input data

In [ ]:
# Checking input data contents
os.listdir(data_path)

In [ ]:
# Defining input images and raw annotations path
img_path=os.path.join(data_path, 'images')
raw_annotations_path=os.path.join(data_path, 'faces.csv')

In [ ]:
# Creating a list of all images
face_list=os.listdir(img_path)

In [ ]:
face_list[:5]

In [ ]:
data_len=len(face_list)
data_len

##### So, there are total 2204 images. Let's shuffle them

In [ ]:
random.shuffle(face_list)
# Checking if they are shuffled
face_list[:5]

##### Defining the train, validation and test split as 80%, 10% and 10% respectively

In [ ]:
train_split=0.8
val_split=0.1
test_split=0.1

#### Separating input data on train, validation and test sets

In [ ]:
imgtrain_list=face_list[:int(data_len*train_split)]
imgval_list=face_list[int(data_len*train_split):int(data_len*(train_split+val_split))]
imgtest_list=face_list[int(data_len*(train_split+val_split)):]

In [ ]:
imgtest_list[:5] # first five images in test set

In [ ]:
# Checking the size of train, validation and test dataset
len(imgtrain_list), len(imgval_list), len(imgtest_list)

#### YOLOv8 requires text file for every image. The text file should have the same name as the image file, only the image extension should be replaced by text extension. This text file contains the bounding box information of the objects in the corresponding image

In [ ]:
# function to extract basename from a file and add a different extension to it. 
def change_extension(file):
    basename=os.path.splitext(file)[0]
    filename=basename+extension
    return filename

##### Creating the lists of text files corresponding to the images in each of the sets

In [ ]:
labeltrain_list = list(map(change_extension, imgtrain_list)) 
labelval_list = list(map(change_extension, imgval_list)) 
labeltest_list = list(map(change_extension, imgtest_list)) 

In [ ]:
# Checking if the list of text files are created correctly 
len(labeltrain_list), len(labelval_list), len(labeltest_list)

In [ ]:

labeltest_list[:5] # matches with the first five images of test set

#### Reading the annotations file

In [ ]:
raw_annotations=pd.read_csv(raw_annotations_path)
raw_annotations

##### The raw annotations contain the diagonal points of the bounding box. YOLOv8 expects the bounding box information in the form of centre coordinates, width and height of the bounding box. So let's transform the information in the required format.

In [ ]:
raw_annotations['x_centre']=0.5*(raw_annotations['x0']+raw_annotations['x1'])
raw_annotations['y_centre']=0.5*(raw_annotations['y0']+raw_annotations['y1'])
raw_annotations['bb_width']=raw_annotations['x1']-raw_annotations['x0']
raw_annotations['bb_height']=raw_annotations['y1']-raw_annotations['y0']
raw_annotations

##### Also the dimensions of bounding box are to be normalised with respect to image width and height

In [ ]:
raw_annotations['xcentre_scaled']=raw_annotations['x_centre']/raw_annotations['width']
raw_annotations['ycentre_scaled']=raw_annotations['y_centre']/raw_annotations['height']
raw_annotations['width_scaled']=raw_annotations['bb_width']/raw_annotations['width']
raw_annotations['height_scaled']=raw_annotations['bb_height']/raw_annotations['height']
raw_annotations

In [ ]:
len(raw_annotations['image_name'].unique())

##### So we have 2204 unique labels. The dataset is consistent.

# Label files creation

##### Creating a text file for every image with the bounding box information in correct format. The correct format for each bounding box is as follows:
##### class_id &nbsp; x_centre &nbsp; y_centre &nbsp; width &nbsp; height
##### This is for every single bounding box. So, if there are multiple objects to detect in one image, there will be as many lines 

In [ ]:
# Getting all unique images
imgs=raw_annotations.groupby('image_name') 

In [ ]:
for image in imgs:
    img_df=imgs.get_group(image[0])
    basename=os.path.splitext(image[0])[0]
    txt_file=basename+extension
    filepath=os.path.join(labels_path, txt_file)
    lines=[]
    i=1
    for index,row in img_df.iterrows():
        if i!=len(img_df):
            line=str(class_id)+bs+str(row['xcentre_scaled'])+bs+str(row['ycentre_scaled'])+bs+str(row['width_scaled'])+bs+str(row['height_scaled'])+newline
            lines.append(line)
        else:
            line=str(class_id)+bs+str(row['xcentre_scaled'])+bs+str(row['ycentre_scaled'])+bs+str(row['width_scaled'])+bs+ str(row['height_scaled'])
            lines.append(line)
        i=i+1
    with open(filepath, 'w') as file:
        file.writelines(lines)
        

In [ ]:
# Checking the labels directory
os.listdir(labels_path)[:5]

##### Let's check what are the contents of any random label file created

In [ ]:
random_file=os.path.join(labels_path, os.listdir(labels_path)[4])
with open (random_file, 'r') as f:
    content=f.read()
content

##### This shows the bounding box data for this image and the order is same as mentioned earlier for every single object present

In [ ]:
def_size=640 # Image size for YOLOv8

In [ ]:
len(os.listdir(labels_path)) # Verifying all labels are created

##### Writing functions to move label files and copy images from their source to train, validation and test directories

In [ ]:
# function to move files from source to detination
def move_files(data_list, source_path, destination_path):
    i=0
    for file in data_list:
        filepath=os.path.join(source_path, file)
        dest_path=os.path.join(data_path, destination_path)
        if not os.path.isdir(dest_path):
            os.makedirs(dest_path)
        shutil.move(filepath, dest_path)
        i=i+1
    print("Number of files transferred:", i)

In [ ]:
# function to resize the images and copy the resized image to destination
def move_images(data_list, source_path, destination_path):
    i=0
    for file in data_list:
        filepath=os.path.join(source_path, file)
        dest_path=os.path.join(data_path, destination_path)
        
        if not os.path.isdir(dest_path):
            os.makedirs(dest_path)
        finalimage_path=os.path.join(dest_path, file)
        img_resized=cv2.resize(cv2.imread(filepath), (def_size, def_size))
        cv2.imwrite(finalimage_path, img_resized)
        i=i+1
    print("Number of files transferred:", i)


#### Moving images from source to train, validation and test directories

In [ ]:
move_images(imgtrain_list, img_path, imgtrainpath)

In [ ]:
move_images(imgval_list, img_path, imgvalpath)

In [ ]:
move_images(imgtest_list, img_path, imgtestpath)

#### Moving labels from source to train, validation and test directories

In [ ]:
move_files(labeltrain_list, labels_path, labeltrainpath)

In [ ]:
move_files(labelval_list, labels_path, labelvalpath)

In [ ]:
move_files(labeltest_list, labels_path, labeltestpath)

##### Checking if all the label files are moved

In [ ]:
len(os.listdir(labels_path)) 

In [ ]:
shutil.rmtree(labels_path) # removing labels path as it is empty

# Creating config file

#### Below is the format of config file for YOLOv8

train: images/train  # train images <br>
val: images/val  # val images <br>
test:  # test images (optional) <br>
<br>
names: <br>
  0: person <br>
  1: bicycle <br>
  2: car <br>
  ... <br>
  77: teddy bear <br>
  78: hair drier <br>
  79: toothbrush


##### Creating and writing the config file in the above format

In [ ]:
ln_1='# Train/val/test sets'+newline
ln_2='train: ' +"'"+imgtrainpath+"'"+newline
ln_3='val: ' +"'" + imgvalpath+"'"+newline
ln_4='test: ' +"'" + imgtestpath+"'"+newline
ln_5=newline
ln_6='# Classes'+newline
ln_7='names:'+newline
ln_8='  0: face'
config_lines=[ln_1, ln_2, ln_3, ln_4, ln_5, ln_6, ln_7, ln_8]

In [ ]:
# Creating path for config file
config_path=os.path.join(curr_path, 'config.yaml')
config_path

In [ ]:
# Writing config file
with open(config_path, 'w') as f:
    f.writelines(config_lines)

# Image Visualisation

#### Let's write a function to obtain bounding box coordinates from text label files.

In [ ]:
# function to obtain bounding box  coordinates from text label files
def get_bbox_from_label(text_file_path):
    bbox_list=[]
    with open(text_file_path, "r") as file:
        for line in file:
            _,x_centre,y_centre,width,height=line.strip().split(" ")
            x1=(float(x_centre)+(float(width)/2))*def_size
            x0=(float(x_centre)-(float(width)/2))*def_size
            y1=(float(y_centre)+(float(height)/2))*def_size
            y0=(float(y_centre)-(float(height)/2))*def_size
            
            vertices=np.array([[int(x0), int(y0)], [int(x1), int(y0)], 
                               [int(x1),int(y1)], [int(x0),int(y1)]])
#             vertices=vertices.reshape((-1,1,2))
            bbox_list.append(vertices)      
            
    return tuple(bbox_list)

#### Drawing bouding box around faces in some randomly selected images in training dataset using training labels 

In [ ]:
# defining red color in RGB to draw bounding box
red=(255,0,0) 

In [ ]:
plt.figure(figsize=(30,30))
for i in range(1,8,2):
    k=random.randint(0, len(imgtrain_list)-1)
    img_path=os.path.join(imgtrainpath, imgtrain_list[k])
    label_path=os.path.join(labeltrainpath, labeltrain_list[k])
    bbox=get_bbox_from_label(label_path)
    image=cv2.imread(img_path)
    image_copy=copy.deepcopy(image)
    ax=plt.subplot(4, 2, i)
    plt.imshow(image) # displaying image
    plt.xticks([])
    plt.yticks([])
    cv2.drawContours(image_copy, bbox, -1, red, 2) # drawing bounding box on copy of image
    ax=plt.subplot(4, 2, i+1)
    plt.imshow(image_copy) # displaying image with bounding box
    plt.xticks([])
    plt.yticks([])

##### These images validate the created text file labels and show that we are good to go for training

# Training

#### Let's install ultralytics and use YOLOv8 to detect faces

In [ ]:
# Installing ultralytics
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

In [ ]:
# Using YOLO's ptetrained model architecture and weights for training
model=YOLO('yolov8n.yaml').load('yolov8n.pt')

In [ ]:
# Training the model
results=model.train(data=config_path, epochs=100, resume=True, iou=0.5, conf=0.001)

Results can be converted to a zip file using the following command which is commented right now. This zip file can be downloaded later if results are to be analysed locally

In [ ]:
# !zip -r results.zip /kaggle/working/runs/detect/train

#### mAP50 is the average precision value obtained by model at 50% IoU. This is the default metric used by YOLOv8 for object detection tasks.

#### Let's see how the training progressed with epochs by visualizing the plots

In [ ]:
plt.figure(figsize=(30,30))
trainingresult_path=os.path.join(curr_path, 'runs', 'detect', 'train')
results_png=cv2.imread(os.path.join(trainingresult_path,'results.png'))
plt.imshow(results_png)

#### All losses- Box loss, class loss, dfl loss are decreasing with epochs.
#### All metrics- Precision, Recall, mAP50 and mAP50-95 are increasing with epochs

#### Let's check the model performance on training, validation and test datasets

# Model Performance 

##### Let's write functions for evaluating model metrics and displaying plots

In [ ]:
# function for evaluating model metrics map50
def evaluate_map50(trainedmodel, data_path, dataset='val'):
    metrics=trainedmodel.val(data=data_path, split=dataset)
    map50=round(metrics.box.map50, 3)
    print("The mAP of model on {0} dataset is {1}".format(dataset,map50))
    return metrics, map50

In [ ]:
# function for displaying plots created by YOLO
def display_curves(root_path):
    plt.figure(figsize=(50,50))
    
    #displaying p curve
    p_curve=cv2.imread(os.path.join(root_path,'P_curve.png'))
    ax=plt.subplot(5,1,1)
    plt.imshow(p_curve)
    
    #displaying r curve
    r_curve=cv2.imread(os.path.join(root_path,'R_curve.png'))
    ax=plt.subplot(5,1,2)
    plt.imshow(r_curve)
    
    #displaying pr curve
    pr_curve=cv2.imread(os.path.join(root_path,'PR_curve.png'))
    ax=plt.subplot(5,1,3)
    plt.imshow(pr_curve)
    
    #displaying f1 curve
    f1_curve=cv2.imread(os.path.join(root_path,'F1_curve.png'))
    ax=plt.subplot(5,1,4)
    plt.imshow(f1_curve)
    
    #displaying confusion matrix
    confusion_matrix=cv2.imread(os.path.join(root_path,'confusion_matrix.png'))
    ax=plt.subplot(5,1,5)
    plt.imshow(confusion_matrix)
    

In [ ]:
# Evaluating train metrics
train_metrics, train_map50=evaluate_map50(model, config_path, dataset='train')

##### Path storing model's performance on training dataset

In [ ]:
train_path=os.path.join(curr_path, 'runs', 'detect', 'val') #val is a misnomer, it is actually measuring validation on training dataset

In [ ]:
# Display plots on training data
display_curves(train_path)

In [ ]:
# Evaluating val metrics
val_metrics, val_map50=evaluate_map50(model, config_path, dataset='val')

##### Path storing model's performance on validation dataset

In [ ]:
val_path=os.path.join(curr_path, 'runs', 'detect', 'val2') 

In [ ]:
# Display plots on validation data
display_curves(val_path)

In [ ]:
# Evaluating test metrics
test_metrics, test_map50=evaluate_map50(model, config_path, dataset='test')

##### Path storing model's performance on test dataset

In [ ]:
test_path=os.path.join(curr_path, 'runs', 'detect', 'val3') #val3 is a misnomer, it is actually measuring validation on test dataset

In [ ]:
# Display plots on test data
display_curves(test_path) 

# Visualizing model's performance on random test images

In [ ]:
plt.figure(figsize=(60,60))
m=random.randint(0, 150) # Selecting random image number
for i in range(1,8,2):
    test_image=os.path.join(imgtestpath, os.listdir(imgtestpath)[m])
    ax=plt.subplot(4,2,i)
    
    # Display actual image
    plt.imshow(cv2.imread(test_image)) 
    plt.xticks([])
    plt.yticks([])
    plt.title("Actual image", fontsize = 40)
    
    # Predict 
    res = model(test_image)
    res_plotted = res[0].plot()
    ax=plt.subplot(4,2,i+1)
    
    # Display image with predictions
    plt.imshow(res_plotted)
    plt.title("Image with predictions", fontsize = 40)
    plt.xticks([])
    plt.yticks([])
    m=m+1

#### So,we see that model detects faces quite nicely. With more data and more training, it can do better